In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import warnings; warnings.filterwarnings("ignore")
import statsmodels.api as sm
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
import matplotlib.pyplot as plt
# plt.style.use('fivethirtyeight')
import seaborn as sns
import autograd.numpy as np
import pandas as pd
import time
from sklearn.neighbors import KernelDensity

In [2]:
tumor_type = "ACC"

In [3]:
TCGA_data      =  './data/Simulated_TCGA_data.tsv'
TCGA_metadata  =  './data/Simulated_TCGA_metadata.tsv'
gene_list      =  './data/Genes_of_Interest.txt'

In [4]:
expression_data  =  pd.read_csv(TCGA_data, sep='\t',index_col=False)
clinical_data    =  pd.read_csv(TCGA_metadata, sep='\t',index_col=False)

In [5]:
alldata = pd.merge(expression_data, clinical_data,on='SampleID', how='left')
df  = alldata.loc[alldata['CancerType'] == tumor_type]

In [6]:
genes = open(gene_list).read().split()

In [7]:
#arr = alldata.CancerType.astype('str').unique()
#cancerlist = arr.tolist()
cancerlist = alldata.CancerType.astype('str').unique().tolist()
print(cancerlist)

['ACC', 'BLCA', 'LGG', 'CESC']


In [8]:
print(alldata.shape, "- - - >>>", df.shape)

(2000, 11) - - - >>> (500, 11)


In [9]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
new_df = df.select_dtypes(include=numerics)

In [10]:
arr = alldata.CancerType.astype('str').unique(); cancerlist = arr.tolist() 
print(cancerlist)

['ACC', 'BLCA', 'LGG', 'CESC']


In [11]:
from scipy.stats import kendalltau, pearsonr, spearmanr

def kendall_pval(x,y):
        return kendalltau(x,y)[1]
    
def pearsonr_pval(x,y):
        return pearsonr(x,y)[1]
    
def spearmanr_pval(x,y):
        return spearmanr(x,y)[1]

In [12]:
new_df.head()

,Gene_A,Gene_B,Gene_C,Gene_D,Gene_E,Gene_F,Gene_G,T,E
0,6.6673,5.7691,9.1580,7.0988,6.7225,9.4109,4.5289,289,1
1,3.8712,4.7949,5.5913,7.6976,9.7312,2.2967,5.2969,245,1
2,5.3227,8.8052,8.5147,8.8254,9.7246,9.5673,6.7407,532,1
3,6.6650,3.1806,8.1542,3.1974,1.5425,8.9108,4.9463,441,1
4,6.0273,2.0288,3.0588,3.2669,3.4156,7.2928,8.5980,894,1


In [13]:
r, p = pearsonr(alldata["Gene_A"], alldata["Gene_B"])
print(r)
print(p)

0.5520817180748576
6.030445876619964e-160


In [14]:
r, p = pearsonr(df['Gene_A'], df['Gene_B'])
print(r); print(p)

0.30278326562057095
4.6292565190255645e-12


In [27]:
df00 = pd.DataFrame(columns = ["CancerType", "GeneID", "pearsonr","p_value"])

count_i = 1
for i in cancerlist:
    df  = alldata.loc[alldata['CancerType'] == i]
    #print(i)
    alldata0 = df[[*genes]]
    #print(alldata0.head())
    data = pd.DataFrame()
    count_j = 0
    for j in range(len(alldata0.columns)):
        a = alldata0.columns[1]; b = alldata0.columns[j]
        r, p = pearsonr(alldata0[a], alldata0[b])
        r = str(round(r,3)); p_value = str('{:0.2e}'.format(p))
        print(i +"\t"+ b +"\t"+ r +"\t"+p_value, end = "\n")
        to_append = [[i, b, r, p_value]]
        df00 = df00.append(pd.DataFrame(to_append,
                                        columns = ["CancerType", "GeneID", "pearsonr","p_value"]),
                                        ignore_index = True)
        df00.to_csv("df.csv", sep="\t", index=False)
        count_j += 1
        time.sleep(3)

ACC	Gene_A	0.303	4.63e-12
ACC	Gene_B	1.0	0.00e+00
ACC	Gene_C	0.329	4.47e-14


KeyboardInterrupt: 

In [16]:
df000.head()

,CancerType,GeneID,pearsonr,p_value
0,ACC,Gene_A,0.303,4.63e-12
1,ACC,Gene_B,1.000,0.00e+00
2,ACC,Gene_C,0.329,4.47e-14
3,ACC,Gene_D,0.393,6.04e-20
4,ACC,Gene_E,0.414,3.77e-22


In [17]:
print(df000)

   CancerType  GeneID  pearsonr   p_value
0         ACC  Gene_A     0.303  4.63e-12
1         ACC  Gene_B     1.000  0.00e+00
2         ACC  Gene_C     0.329  4.47e-14
3         ACC  Gene_D     0.393  6.04e-20
4         ACC  Gene_E     0.414  3.77e-22
5         ACC  Gene_F     0.372  7.26e-18
6         ACC  Gene_G     0.448  4.65e-26
7        BLCA  Gene_A     0.040  3.71e-01
8        BLCA  Gene_B     1.000  0.00e+00
9        BLCA  Gene_C     0.117  8.66e-03
10       BLCA  Gene_D     0.084  5.88e-02
11       BLCA  Gene_E     0.087  5.16e-02
12       BLCA  Gene_F     0.086  5.48e-02
13       BLCA  Gene_G     0.045  3.19e-01
14        LGG  Gene_A     0.013  7.78e-01
15        LGG  Gene_B     1.000  0.00e+00
16        LGG  Gene_C    -0.065  1.45e-01
17        LGG  Gene_D    -0.031  4.87e-01
18        LGG  Gene_E     0.032  4.73e-01
19        LGG  Gene_F     0.076  9.03e-02
20        LGG  Gene_G     0.066  1.38e-01
21       CESC  Gene_A     0.239  6.33e-08
22       CESC  Gene_B     1.000  0